# 🏷️ Project: Tag, You're Nash!
## Analyzing the Impact of Reward Structures on Strategic Stability in MARL

**Submitted by:** Omer Toledano & Eliya Naomi Aharon

**Course:** Multi-Agent Systems (MAS) | Ben-Gurion University

In [1]:
# Install necessary libraries for the environment and algorithms
! pip install pettingzoo[mpe] supersuit stable-baselines3 pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.1/952.1 kB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 17.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.5/852.5 kB 4.3 MB/s eta 0:00:00


In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import multiprocessing

# PettingZoo & SuperSuit
import supersuit as ss
from pettingzoo.mpe import simple_tag_v3
from pettingzoo.utils.wrappers import BaseParallelWrapper

# Stable Baselines3
from stable_baselines3 import PPO

# Set up the device
device = "cuda" if torch.cuda.is_available() else "cpu"
num_cores = max(1, multiprocessing.cpu_count() - 2)
os.environ["SDL_VIDEODRIVER"] = "dummy"

/tmp/ipykernel_3308228/1632111876.py:11: DeprecationWarning: The environment `pettingzoo.mpe` has been moved to `mpe2` and will be removed in a future release.Please update your imports.
  from pettingzoo.mpe import simple_tag_v3


### Research Question 2
**RQ2:** *How do the learning dynamics change when the reward structure shifts from pure competition to partial cooperation?*

### 🧪 Methodology
To answer this, we introduce a **Reward Sharing Mechanism** controlled by a mixing parameter $\alpha$ (alpha):
* **$\alpha = 1.0$ (Pure Competition):** Agents behave selfishly (Baseline).
* **$\alpha = 0.5$ (Partial Cooperation):** Agents share 50% of the reward with the team.
* **$\alpha = 0.0$ (Full Cooperation):** Agents share 100% of the reward (Team Spirit).

We hypothesize that shifting $\alpha$ from `1.0` to `0.5` will break the "stalling" equilibrium observed in earlier experiments and encourage coordinated encirclement strategies.

Set up the enviroment and helper functions

In [4]:
class RewardSharingWrapper(BaseParallelWrapper):
    """
    A wrapper to modify the reward structure of the predators.
    
    Parameters:
    - env: The PettingZoo parallel environment.
    - alpha (float): The cooperation factor (0.0 to 1.0).
        * alpha = 1.0: Pure Individual Reward (Competition).
        * alpha = 0.0: Pure Shared Reward (Full Cooperation).
        * alpha = 0.5: Mixed Strategy.
    """
    def __init__(self, env, alpha=1.0):
        super().__init__(env)
        self.alpha = alpha
        # Identify which agents are predators (adversaries)
        self.group_agents = [a for a in env.possible_agents if "adversary" in a]

    def step(self, actions):
        # Perform the standard environment step
        obs, rewards, terminations, truncations, infos = self.env.step(actions)
        
        # Calculate the total reward achieved by the predator group
        group_reward_sum = sum(rewards[a] for a in self.group_agents)
        
        # Redistribute rewards based on alpha
        new_rewards = rewards.copy()
        
        for agent in self.group_agents:
            individual_r = rewards[agent]
            
            # Formula: (alpha * Own_Reward) + ((1-alpha) * Group_Total)
            # If alpha is 1, they only care about themselves.
            # If alpha is 0, they only care about the team sum.
            shared_r = (self.alpha * individual_r) + ((1 - self.alpha) * group_reward_sum)
            new_rewards[agent] = shared_r
            
        return obs, new_rewards, terminations, truncations, infos

def create_env(alpha=1.0):
    """
    Creates a vectorized environment for faster training.
    
    Args:
        alpha (float): The cooperation parameter.
        num_envs (int): How many games to play in parallel (speed up).
    """
    # 1. Initialize the base environment configuration
    env = simple_tag_v3.parallel_env(
        num_good=1, 
        num_adversaries=3, 
        num_obstacles=2, 
        max_cycles=50, 
        continuous_actions=False
    )
    
    # 2. Apply the Reward Sharing Wrapper (RQ2 Implementation)
    env = RewardSharingWrapper(env, alpha=alpha)
    
    # 3. Apply SuperSuit wrappers for SB3 compatibility
    env = ss.pad_observations_v0(env) # Ensure equal observation size
    env = ss.pad_action_space_v0(env) # Ensure equal action size
    env = ss.pettingzoo_env_to_vec_env_v1(env) # Convert to Vector Env

    env = ss.concat_vec_envs_v1(
        env, 
        num_vec_envs=1,  
        num_cpus=num_cores,      
        base_class="stable_baselines3"
    )
    return env

def run_experiment(alpha, total_timesteps=50000, rounds=3):
    """
    Runs the full adversarial training experiment for a specific alpha.
    
    Args:
        alpha: Cooperation level.
        total_timesteps: Steps per training round (Higher = Better learning).
        rounds: Number of alternating training cycles.
    """
    print(f"\n{'='*60}")
    print(f"🚀 STARTING EXPERIMENT | Alpha: {alpha} ")
    print(f"{'='*60}")
    
    # 1. Create separate environments for Predator and Prey models
    # We use 8 parallel environments to speed up data collection (8x faster)
    env_pred = create_env(alpha=alpha)
    env_prey = create_env(alpha=alpha)
    
    # 2. Initialize PPO Models
    # Note: 'device=device' ensures usage of GPU if available
    predator_model = PPO("MlpPolicy", env_pred, verbose=0, learning_rate=0.0003)
    prey_model = PPO("MlpPolicy", env_prey, verbose=0, learning_rate=0.0003)
    
    # 3. Adversarial Training Loop (Self-Play)
    for i in range(rounds):
        start_time = time.time()
        print(f"\n🔄 Round {i+1}/{rounds} in progress...")
        
        # Train Predators
        predator_model.learn(total_timesteps=total_timesteps)
        print(f"   > Predators trained ({total_timesteps} steps)")
        
        # Train Prey (Adapting to new predators)
        prey_model.learn(total_timesteps=total_timesteps)
        print(f"   > Prey trained ({total_timesteps} steps)")
        
        elapsed = time.time() - start_time
        print(f"   ⏱️ Round finished in {elapsed:.2f} seconds.")
    
    # Save Models
    pred_path = f"./models/predator_alpha_{alpha}"
    prey_path = f"./models/prey_alpha_{alpha}"
    
    # Ensure directory exists
    os.makedirs("models", exist_ok=True)
    
    predator_model.save(pred_path)
    prey_model.save(prey_path)
    
    print(f"\n✅ Experiment Complete. Models saved to 'models/' folder.")

Run the experiment

In [ ]:
# # Run baseline (Pure Competition)
# run_experiment(alpha=1.0, total_timesteps=100000, rounds=10)

# Run experimental (Partial Cooperation)
# run_experiment(alpha=0.5, total_timesteps=100000, rounds=10)


🚀 STARTING EXPERIMENT | Alpha: 0.5 

🔄 Round 1/10 in progress...
   > Predators trained (100000 steps)
   > Prey trained (100000 steps)
   ⏱️ Round finished in 218.32 seconds.

🔄 Round 2/10 in progress...
   > Predators trained (100000 steps)
   > Prey trained (100000 steps)
   ⏱️ Round finished in 218.37 seconds.

🔄 Round 3/10 in progress...
   > Predators trained (100000 steps)
   > Prey trained (100000 steps)
   ⏱️ Round finished in 218.14 seconds.

🔄 Round 4/10 in progress...
   > Predators trained (100000 steps)
   > Prey trained (100000 steps)
   ⏱️ Round finished in 218.82 seconds.

🔄 Round 5/10 in progress...
   > Predators trained (100000 steps)
   > Prey trained (100000 steps)
   ⏱️ Round finished in 218.71 seconds.

🔄 Round 6/10 in progress...
   > Predators trained (100000 steps)
   > Prey trained (100000 steps)
   ⏱️ Round finished in 268.38 seconds.

🔄 Round 7/10 in progress...
   > Predators trained (100000 steps)
   > Prey trained (100000 steps)
   ⏱️ Round finished in

Evaluaton

In [ ]:
def evaluate_model(alpha, episodes=20, max_cycles=50):
    """
    Runs a test simulation and logs detailed metrics: 
    Captures, Collisions, Episode Sums, and Final Averages.
    """
    model_path = f"models/predator_alpha_{alpha}"
        
    env = create_env(alpha=alpha)
    pred_model = PPO.load(model_path)
    
    all_episode_rewards = []
    total_captures = 0
    total_collisions = 0
    
    log_file = f"alpha_{alpha}_detailed_evaluation.txt"
    
    with open(log_file, "w") as f:
        f.write(f"DETAILED EVALUATION LOG | Alpha: {alpha}\n")
        f.write(f"Timestamp: {time.ctime()}\n")
        f.write("-" * 50 + "\n")

    print(f"\n🚀 Starting Evaluation for Alpha {alpha}...")

    for ep in range(episodes):
        obs = env.reset()
        ep_reward = 0
        ep_captures = 0
        ep_collisions = 0
        
        for step in range(max_cycles):
            action, _ = pred_model.predict(obs)
            obs, rewards, dones, infos = env.step(action)
            
            # 1. Sum rewards for the step [cite: 37]
            predator_rewards = [rewards[a] for a in env.possible_agents if "adversary" in a]
            prey_rewards = [rewards[a] for a in env.possible_agents if "good" in a]

            step_reward = np.sum(predator_rewards)
            ep_reward += step_reward
            
            # 2. Track Captures (Heuristic based on positive reward) [cite: 37]
            if step_reward > 8: 
                ep_captures += 1
            
            # 3. Track Collisions (Fixed for list-based infos) [cite: 33, 38]
            # If infos is a list (VectorEnv), we look at the first env's info
            current_info = infos[0] if isinstance(infos, list) else infos
            
            # Safely iterate through agent info to count collisions
            if isinstance(current_info, dict):
                for agent_data in current_info.values():
                    if isinstance(agent_data, dict) and agent_data.get("collision", False):
                        ep_collisions += 1

        all_episode_rewards.append(ep_reward)
        total_captures += ep_captures
        total_collisions += ep_collisions
        
        # Log individual episode results
        with open(log_file, "a") as f:
            f.write(f"Episode {ep+1}: Total Reward: {ep_reward:.2f} | "
                    f"Captures: {ep_captures} | Collisions: {ep_collisions}\n")

    # Final Statistics for Alpha stability analysis [cite: 6, 43]
    total_sum_all_episodes = np.sum(all_episode_rewards)
    avg_reward = np.mean(all_episode_rewards)
    avg_captures = total_captures / episodes
    avg_collisions = total_collisions / episodes

    with open(log_file, "a") as f:
        f.write("-" * 50 + "\n")
        f.write(f"FINAL STATISTICS FOR ALPHA {alpha}:\n")
        f.write(f"Total Sum (All Episodes): {total_sum_all_episodes:.2f}\n")
        f.write(f"Average Reward: {avg_reward:.2f}\n")
        f.write(f"Total Captures: {total_captures} (Avg: {avg_captures:.2f}/ep)\n")
        f.write(f"Total Collisions: {total_collisions} (Avg: {avg_collisions:.2f}/ep)\n")
        f.write("-" * 50 + "\n")

    print(f"✅ Evaluation complete for Alpha {alpha}. Logs saved.")
    return avg_reward

# Run Evaluation
print("\n📊 EVALUATION RESULTS:")
res = []
for i in [0.5, 1.0]:
    a = i #/ 10
    score = evaluate_model(a)
    res.append((i, score))

[print(f"Average Reward for Alpha = {a}: {score:.2f}") for a, score in res]


📊 EVALUATION RESULTS:

🚀 Starting Evaluation for Alpha 0.5...
✅ Evaluation complete for Alpha 0.5. Logs saved.

🚀 Starting Evaluation for Alpha 1.0...
✅ Evaluation complete for Alpha 1.0. Logs saved.
Average Reward for Alpha = 0.5: 917.35
Average Reward for Alpha = 1.0: 187.27


[None, None]